In [88]:
import os

import numpy as np
import torch
import tensorflow as tf
keras = tf.keras
K = keras.backend
from model import BiSeNet

from resnet import Resnet18

In [3]:
def restore_pytorch_model(save_path=os.path.join("res/cp", "79999_iter.pth")):
    n_classes = 19
    
    net = BiSeNet(n_classes=n_classes)
#     net.cuda()
    
    net.load_state_dict(torch.load(save_path))
#     net.eval()

    return net

In [6]:
model_pytorch = restore_pytorch_model()

In [7]:
model_pytorch

BiSeNet(
  (cp): ContextPath(
    (resnet): Resnet18(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, a

## Convert Resnet18

In [300]:
resnet18_url = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return keras.layers.Conv2D(
#         in_planes,
        out_planes,
        kernel_size=3, 
        strides=stride,
        padding="same", 
        use_bias=False
    )


class BasicBlock(keras.layers.Layer):
    def __init__(self, in_chan, out_chan, stride=1):
        super(BasicBlock, self).__init__()
        
        self.conv1 = conv3x3(in_chan, out_chan, stride)
        self.bn1 = keras.layers.BatchNormalization()
        self.conv2 = conv3x3(out_chan, out_chan)
        self.bn2 = keras.layers.BatchNormalization()
#         self.relu = nn.ReLU(inplace=True)
        self.relu = keras.activations.relu
        self.downsample = None
        if in_chan != out_chan or stride != 1:
            self.downsample = keras.models.Sequential()
            self.downsample.add(
                keras.layers.Conv2D(
#                     in_chan, 
                    out_chan,
                    kernel_size=1, 
                    strides=stride, 
                    use_bias=False
                )
            )
            self.downsample.add(keras.layers.BatchNormalization())

    def call(self, x):
        residual = self.conv1(x)
        residual = self.relu(self.bn1(residual))
        residual = self.conv2(residual)
        residual = self.bn2(residual)

        shortcut = x
        if self.downsample is not None:
            shortcut = self.downsample(x)

        out = shortcut + residual
        out = self.relu(out)
        return out
    
    # TODO
    def load_weigths_pytorch(self, state_dict):
        layers = [
            self.conv1,
            self.bn1,
            self.conv2,
            self.bn2,
        ]
        pth_layer_names = [
            "conv1",
            "bn1",
            "conv2",
            "bn2",
        ]
        
        for layer, pth_layer_name in zip(layers, pth_layer_names):
            layer_state_dict = filter_dict_by_name(pth_layer_name, state_dict)
            layer_state_dict = strip_dict_key_prefix(layer_state_dict)
            
            if isinstance(layer, keras.layers.Conv2D):
                weights_conversion_fn = conv2d_weights_pth2tf
            elif isinstance(layer, keras.layers.BatchNormalization):
                weights_conversion_fn = bn_weights_pth2tf
                
#             print(layer_state_dict)
            layer.set_weights(weights_conversion_fn(layer_state_dict))
    
        if self.downsample is not None:
            for idx, layer in enumerate(self.downsample):
                print(f"idx: {idx}, layer type: {type(layer)}")

                layer_state_dict = filter_dict_by_name(f"downsample.{idx}", state_dict)
                layer_state_dict = strip_dict_key_prefix(layer_state_dict, cut=2)
                
                if isinstance(layer, keras.layers.Conv2D):
                    weights_conversion_fn = conv2d_weights_pth2tf
                elif isinstance(layer, keras.layers.BatchNormalization):
                    weights_conversion_fn = bn_weights_pth2tf
                    
                layer.set_weights(weights_conversion_fn(layer_state_dict))

    
        print("Done loading BasicBlock weights!")


def create_layer_basic(in_chan, out_chan, bnum, stride=1):
#     layers = [BasicBlock(in_chan, out_chan, stride=stride)]
#     for i in range(bnum-1):
#         layers.append(BasicBlock(out_chan, out_chan, stride=1))
#     return nn.Sequential(*layers)
    model = keras.models.Sequential()
    model.add(BasicBlock(in_chan, out_chan, stride=stride))
    for i in range(bnum - 1):
        model.add(BasicBlock(out_chan, out_chan, stride=1))
    return model


class tfResnet18(keras.models.Model):
    def __init__(self):
        super(tfResnet18, self).__init__()
        self.conv1 = keras.layers.Conv2D(
#             3, 
            64, 
            kernel_size=7, 
            strides=2, 
            padding="same",
            use_bias=False
        )
        self.bn1 = keras.layers.BatchNormalization()
        self.maxpool = keras.layers.MaxPool2D(3, strides=2, padding="same")
        self.layer1 = create_layer_basic(64, 64, bnum=2, stride=1)
        self.layer2 = create_layer_basic(64, 128, bnum=2, stride=2)
        self.layer3 = create_layer_basic(128, 256, bnum=2, stride=2)
        self.layer4 = create_layer_basic(256, 512, bnum=2, stride=2)

    def call(self, x):
        x = self.conv1(x)
        x = K.relu(self.bn1(x))
        x = self.maxpool(x)

        x = self.layer1(x)
        feat8 = self.layer2(x) # 1/8
        feat16 = self.layer3(feat8) # 1/16
        feat32 = self.layer4(feat16) # 1/32
        return feat8, feat16, feat32
    
    def load_weigths_pytorch(self, weights_path):
        model_state_dict = torch.load(weights_path)
        
        layers = [
            self.conv1,
            self.bn1,
        ]
        pth_layer_names = [
            "conv1",
            "bn1",
        ]
        
        for layer, pth_layer_name in zip(layers, pth_layer_names):
            layer_state_dict = filter_dict_by_name(pth_layer_name, model_state_dict)
            layer_state_dict = strip_dict_key_prefix(layer_state_dict)
            
            if isinstance(layer, keras.layers.Conv2D):
                weights_conversion_fn = conv2d_weights_pth2tf
            elif isinstance(layer, keras.layers.BatchNormalization):
                weights_conversion_fn = bn_weights_pth2tf
                
#             print(layer_state_dict)
            layer.set_weights(weights_conversion_fn(layer_state_dict))
            
            
    
        print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        for idx, layer in enumerate(self.layer1.layers):
            print(f"idx: {idx}, layer type: {type(layer)}")
            print(isinstance(layer, BasicBlock))
            
            layer_state_dict = filter_dict_by_name(f"layer1.{idx}", model_state_dict)
            layer_state_dict = strip_dict_key_prefix(layer_state_dict, cut=2)
            print(layer_state_dict)
            layer.load_weigths_pytorch(layer_state_dict)

In [301]:
def get_layers(param_names):
    layers = []
    
    for name in param_names:
        lname = ".".join(name.split(".")[:-1])
        
        if lname not in layers:
            layers.append(lname)
        
    return layers


def filter_dict_by_name(name, model_state_dict):
    return dict(filter(lambda t: t[0].startswith(name),  model_state_dict.items()))


def conv2d_weights_pth2tf(params_dict):
#     weights = []
    
#     for _, t in params_dict.items():
#         t = t.detach().numpy()
        
#         if t.ndim == 4:    
#             t = np.transpose(t, [2, 3, 1, 0])
            
#         weights.append(t)
    
#     return weights

    weights = []
    
    params_dict = strip_dict_key_prefix(params_dict, -1)
    
    # gamma, beta, moving_mean, moving_variance
    for key in ["weight", "bias"]:
        if key not in params_dict:
            continue
            
        if key == "weight":
            weights.append(np.transpose(params_dict[key].detach().numpy(), [2, 3, 1, 0]))
        else:
            weights.append(params_dict[key].detach().numpy())
    
    return weights


def strip_dict_key_prefix(params_dict, cut=-2):
    return {f"{'.'.join(k.split('.')[cut:])}": v for k, v in params_dict.items()}


def bn_weights_pth2tf(params_dict):
    weights = []
    
    params_dict = strip_dict_key_prefix(params_dict, -1)
    
    # gamma, beta, moving_mean, moving_variance
    for key in ["weight", "bias", "running_mean", "running_var"]:
        weights.append(params_dict[key].detach().numpy())
    
    return weights

In [302]:
net_tf = tfResnet18()
x = tf.random.normal((16, 224, 224, 3))
out = net_tf(x)
print(out[0].shape)
print(out[1].shape)
print(out[2].shape)

(16, 28, 28, 128)
(16, 14, 14, 256)
(16, 7, 7, 512)


In [303]:
net_tf.load_weigths_pytorch("resnet18-5c106cde.pth")

>>>>>>>>>>>>>>>>>>>>>>>>>>>>
idx: 0, layer type: <class '__main__.BasicBlock'>
True
{'conv1.weight': Parameter containing:
tensor([[[[ 5.7593e-02, -9.5114e-02, -2.0272e-02],
          [-7.4556e-02, -7.9931e-01, -2.1284e-01],
          [ 6.5571e-02, -9.6534e-02, -1.2111e-02]],

         [[-6.9944e-03,  1.4266e-02,  5.5824e-04],
          [ 4.1238e-02, -1.6125e-01, -2.3208e-02],
          [ 3.2887e-03,  7.1779e-03,  7.1686e-02]],

         [[-2.3627e-09, -3.9270e-08, -3.2971e-08],
          [ 2.1737e-08,  8.3299e-09,  1.2543e-08],
          [ 1.1382e-08,  8.8096e-09,  1.5506e-08]],

         ...,

         [[-3.6921e-02,  1.8294e-02, -2.9358e-02],
          [-9.8615e-02, -4.3645e-02, -5.2717e-02],
          [-7.9635e-02,  2.9396e-02,  4.1479e-03]],

         [[ 1.6948e-02,  1.3978e-02,  9.6727e-03],
          [ 1.4297e-02, -6.6985e-04, -2.2077e-02],
          [ 1.2398e-02,  3.5454e-02, -2.2320e-02]],

         [[-2.2600e-02, -2.5331e-02, -2.3548e-02],
          [ 6.0860e-02, -9.6779e-02,

In [299]:
net_tf.summary()

Model: "tf_resnet18_65"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1225 (Conv2D)         multiple                  9408      
_________________________________________________________________
batch_normalization_1224 (Ba multiple                  256       
_________________________________________________________________
max_pooling2d_62 (MaxPooling multiple                  0         
_________________________________________________________________
sequential_428 (Sequential)  multiple                  148480    
_________________________________________________________________
sequential_429 (Sequential)  multiple                  526848    
_________________________________________________________________
sequential_431 (Sequential)  multiple                  2102272   
_________________________________________________________________
sequential_433 (Sequential)  multiple               

In [180]:
net = Resnet18()
x = torch.randn(16, 3, 224, 224)
out = net(x)
print(out[0].size())
print(out[1].size())
print(out[2].size())

torch.Size([16, 128, 28, 28])
torch.Size([16, 256, 14, 14])
torch.Size([16, 512, 7, 7])


In [45]:
net

Resnet18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1

In [49]:
state_dict = torch.load("resnet18-5c106cde.pth")

In [54]:
list(state_dict.keys())

['conv1.weight',
 'bn1.running_mean',
 'bn1.running_var',
 'bn1.weight',
 'bn1.bias',
 'layer1.0.conv1.weight',
 'layer1.0.bn1.running_mean',
 'layer1.0.bn1.running_var',
 'layer1.0.bn1.weight',
 'layer1.0.bn1.bias',
 'layer1.0.conv2.weight',
 'layer1.0.bn2.running_mean',
 'layer1.0.bn2.running_var',
 'layer1.0.bn2.weight',
 'layer1.0.bn2.bias',
 'layer1.1.conv1.weight',
 'layer1.1.bn1.running_mean',
 'layer1.1.bn1.running_var',
 'layer1.1.bn1.weight',
 'layer1.1.bn1.bias',
 'layer1.1.conv2.weight',
 'layer1.1.bn2.running_mean',
 'layer1.1.bn2.running_var',
 'layer1.1.bn2.weight',
 'layer1.1.bn2.bias',
 'layer2.0.conv1.weight',
 'layer2.0.bn1.running_mean',
 'layer2.0.bn1.running_var',
 'layer2.0.bn1.weight',
 'layer2.0.bn1.bias',
 'layer2.0.conv2.weight',
 'layer2.0.bn2.running_mean',
 'layer2.0.bn2.running_var',
 'layer2.0.bn2.weight',
 'layer2.0.bn2.bias',
 'layer2.0.downsample.0.weight',
 'layer2.0.downsample.1.running_mean',
 'layer2.0.downsample.1.running_var',
 'layer2.0.downsam

In [55]:
list(net.state_dict().keys())

['conv1.weight',
 'bn1.weight',
 'bn1.bias',
 'bn1.running_mean',
 'bn1.running_var',
 'bn1.num_batches_tracked',
 'layer1.0.conv1.weight',
 'layer1.0.bn1.weight',
 'layer1.0.bn1.bias',
 'layer1.0.bn1.running_mean',
 'layer1.0.bn1.running_var',
 'layer1.0.bn1.num_batches_tracked',
 'layer1.0.conv2.weight',
 'layer1.0.bn2.weight',
 'layer1.0.bn2.bias',
 'layer1.0.bn2.running_mean',
 'layer1.0.bn2.running_var',
 'layer1.0.bn2.num_batches_tracked',
 'layer1.1.conv1.weight',
 'layer1.1.bn1.weight',
 'layer1.1.bn1.bias',
 'layer1.1.bn1.running_mean',
 'layer1.1.bn1.running_var',
 'layer1.1.bn1.num_batches_tracked',
 'layer1.1.conv2.weight',
 'layer1.1.bn2.weight',
 'layer1.1.bn2.bias',
 'layer1.1.bn2.running_mean',
 'layer1.1.bn2.running_var',
 'layer1.1.bn2.num_batches_tracked',
 'layer2.0.conv1.weight',
 'layer2.0.bn1.weight',
 'layer2.0.bn1.bias',
 'layer2.0.bn1.running_mean',
 'layer2.0.bn1.running_var',
 'layer2.0.bn1.num_batches_tracked',
 'layer2.0.conv2.weight',
 'layer2.0.bn2.weigh

In [56]:
net.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['fc.weight', 'fc.bias'])

In [67]:
for k, v in state_dict.items():
    v = v.detach().numpy()
    
    print(f"layer: {k}\t\t\t\tshape: {v.shape}")

layer: conv1.weight				shape: (64, 3, 7, 7)
layer: bn1.running_mean				shape: (64,)
layer: bn1.running_var				shape: (64,)
layer: bn1.weight				shape: (64,)
layer: bn1.bias				shape: (64,)
layer: layer1.0.conv1.weight				shape: (64, 64, 3, 3)
layer: layer1.0.bn1.running_mean				shape: (64,)
layer: layer1.0.bn1.running_var				shape: (64,)
layer: layer1.0.bn1.weight				shape: (64,)
layer: layer1.0.bn1.bias				shape: (64,)
layer: layer1.0.conv2.weight				shape: (64, 64, 3, 3)
layer: layer1.0.bn2.running_mean				shape: (64,)
layer: layer1.0.bn2.running_var				shape: (64,)
layer: layer1.0.bn2.weight				shape: (64,)
layer: layer1.0.bn2.bias				shape: (64,)
layer: layer1.1.conv1.weight				shape: (64, 64, 3, 3)
layer: layer1.1.bn1.running_mean				shape: (64,)
layer: layer1.1.bn1.running_var				shape: (64,)
layer: layer1.1.bn1.weight				shape: (64,)
layer: layer1.1.bn1.bias				shape: (64,)
layer: layer1.1.conv2.weight				shape: (64, 64, 3, 3)
layer: layer1.1.bn2.running_mean				shape: (64,)
lay

In [10]:
class ConvBNReLU(keras.layers.Layer)
    def __init__(self, in_chan, out_chan, ks=3, stride=1, padding=1, *args, **kwargs):
        super(ConvBNReLU, self).__init__()
        
        self.conv = keras.layers.Conv2D(
                out_chan,
                kernel_size = ks,
                strides = stride,
                padding = padding,
                use_bias = False)
        
        self.bn = keras.layers.BatchNormalization()

    def call(self, x):
        x = self.conv(x)
        x = tf.nn.relu(self.bn(x))
        return x

In [11]:
class BiSeNetOutput(keras.layers.Layer):
    def __init__(self, in_chan, mid_chan, n_classes, *args, **kwargs):
        super(BiSeNetOutput, self).__init__()
        
        self.conv = ConvBNReLU(in_chan, 
                               mid_chan, 
                               ks=3, 
                               stride=1, 
                               padding="same")
        self.conv_out = keras.layers.Conv2D(
#                                 mid_chan, 
                                  n_classes, 
                                  kernel_size=1, 
                                  use_bias=False)

    def call(self, x):
        x = self.conv(x)
        x = self.conv_out(x)
        return x

In [12]:
class AttentionRefinementModule(keras.layers.Layer):
    def __init__(self, in_chan, out_chan, *args, **kwargs):
        super(AttentionRefinementModule, self).__init__()
        
        self.conv = ConvBNReLU(in_chan, 
                               out_chan, 
                               ks=3, 
                               stride=1, 
                               padding="same")
        
        self.conv_atten = keras.layers.Conv2D(
#                                     out_chan, 
                                    out_chan, 
                                    kernel_size=1, 
                                    use_bias=False)
        
        self.bn_atten = keras.layers.BatchNormalization()
        self.sigmoid_atten = keras.layers.Activation("sigmoid")
        self.avg_pool2d = keras.layers.GlobalAveragePooling2D()

    def forward(self, x):
        feat = self.conv(x)
#         atten = self.avg_pool2d(feat, feat.size()[2:])
        atten = self.avg_pool2d(feat)
        atten = self.conv_atten(atten)
        atten = self.bn_atten(atten)
        atten = self.sigmoid_atten(atten)
        out = tf.math.muliply(feat, atten)
        return out

In [ ]:
class ContextPath(nn.Module):
    def __init__(self, *args, **kwargs):
        super(ContextPath, self).__init__()
        
        self.resnet = Resnet18()
        self.arm16 = AttentionRefinementModule(256, 128)
        self.arm32 = AttentionRefinementModule(512, 128)
        self.conv_head32 = ConvBNReLU(128, 128, ks=3, stride=1, padding=1)
        self.conv_head16 = ConvBNReLU(128, 128, ks=3, stride=1, padding=1)
        self.conv_avg = ConvBNReLU(512, 128, ks=1, stride=1, padding=0)

    def forward(self, x):
        H0, W0 = x.size()[2:]
        
        feat8, feat16, feat32 = self.resnet(x)
        H8, W8 = feat8.size()[2:]
        H16, W16 = feat16.size()[2:]
        H32, W32 = feat32.size()[2:]

        avg = F.avg_pool2d(feat32, feat32.size()[2:])
        avg = self.conv_avg(avg)
        avg_up = F.interpolate(avg, (H32, W32), mode='nearest')

        feat32_arm = self.arm32(feat32)
        feat32_sum = feat32_arm + avg_up
        feat32_up = F.interpolate(feat32_sum, (H16, W16), mode='nearest')
        feat32_up = self.conv_head32(feat32_up)

        feat16_arm = self.arm16(feat16)
        feat16_sum = feat16_arm + feat32_up
        feat16_up = F.interpolate(feat16_sum, (H8, W8), mode='nearest')
        feat16_up = self.conv_head16(feat16_up)

        return feat8, feat16_up, feat32_up  # x8, x8, x16

In [15]:
class FeatureFusionModule(keras.layers.Layer):
    def __init__(self, in_chan, out_chan, *args, **kwargs):
        super(FeatureFusionModule, self).__init__()
        
        self.convblk = ConvBNReLU(
            in_chan,
            out_chan,
            ks=1,
            stride=1,
            padding="same"
        )
        
        self.conv1 = keras.layers.Conv2d(
#             out_chan,
            out_chan//4,
            kernel_size=1,
            strides=1,
            padding="same",
            use_bias=False
        )
        self.conv2 = keras.layers.Conv2D(
#             out_chan//4,
            out_chan,
            kernel_size=1,
            strides=1,
            padding="same",
            use_bias=False
        )
#         self.relu = nn.ReLU(inplace=True)

    def call(self, inputs):
        fsp, fcp = inputs
        fcat = K.concatenate([fsp, fcp], axis=3)
        
        feat = self.convblk(fcat)
        
        atten = K.pool2d(feat, feat.shape[1:2+1], pool_mode="avg")
        atten = self.conv1(atten)
        atten = K.relu(atten)
        atten = self.conv2(atten)
        atten = K.sigmoid(atten)
        feat_atten = tf.math.multiply(feat, atten)
        feat_out = feat_atten + feat
        return feat_out

In [20]:
class tfBiSeNet(keras.models.Model):
    def __init__(self, n_classes, *args, **kwargs):
        super(tfBiSeNet, self).__init__()
        
        self.cp = ContextPath()
        ## here self.sp is deleted
        
        self.ffm = FeatureFusionModule(256, 256)
        
        self.conv_out = BiSeNetOutput(256, 256, n_classes)
        self.conv_out16 = BiSeNetOutput(128, 64, n_classes)
        self.conv_out32 = BiSeNetOutput(128, 64, n_classes)
        
        self.image_data_format = K.image_data_format()

    def call(self, x):
#         H, W = x.size()[2:]
        H, W = x.shape[1:2+1]
        print(f"H: {H}, W: {W}")
    
        feat_res8, feat_cp8, feat_cp16 = self.cp(x)  # here return res3b1 feature
        
        feat_sp = feat_res8  # use res3b1 feature to replace spatial path feature
        feat_fuse = self.ffm(feat_sp, feat_cp8)

        feat_out = self.conv_out(feat_fuse)
        feat_out16 = self.conv_out16(feat_cp8)
        feat_out32 = self.conv_out32(feat_cp16)

        feat_out = K.resize_images(feat_out, 
                                   H, W, 
                                   data_format=self.image_data_format, 
                                   interpolation='bilinear')
        feat_out16 = K.resize_images(feat_out16, 
                                     H, W, 
                                     data_format=self.image_data_format, 
                                     interpolation='bilinear')
        feat_out32 = K.resize_images(feat_out32, 
                                     H, W, 
                                     data_format=self.image_data_format, 
                                     interpolation='bilinear')
        
        return feat_out, feat_out16, feat_out32

In [21]:
model_tf = tfBiSeNet(n_classes=19)

NameError: name 'ContextPath' is not defined